# Experiment

> Scripts to perform the experiments

In [1]:
#| default_exp experiment

In [2]:
#| export
#| hide
import os
import csv
import pandas as pd
import subprocess
import numpy as np
import shutil
import seaborn as sns
import json
from typing import Dict, Any, List, Optional, Tuple
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

from orbit_generation.constants import ORBIT_CLASS_DF

In [3]:
#| hide
from fastcore.test import test_eq

## Setup

In [4]:
#| export
def setup_new_experiment(params: Dict[str, Any],              # Dictionary of parameters for the new experiment.
                         experiments_folder: str,             # Path to the folder containing all experiments.
                         json_file: Optional[str] = None      # Optional path to the JSON file tracking experiment parameters.
                        ) -> str:                             # The path to the newly created experiment folder.
    """
    Sets up a new experiment by creating a new folder and updating the JSON file with experiment parameters.
    """
    # Ensure the experiments folder exists
    if not os.path.exists(experiments_folder):
        os.makedirs(experiments_folder)

    # Default JSON file to 'experiments.json' in the experiments_folder if not provided
    if json_file is None:
        json_file = os.path.join(experiments_folder, 'experiments.json')

    # Load existing experiments from the JSON file if it exists
    if os.path.isfile(json_file):
        with open(json_file, mode='r') as file:
            experiments = json.load(file)
    else:
        experiments = []

    # Check if the parameters already exist in the JSON file
    for experiment in experiments:
        if all(experiment['parameters'].get(key) == value for key, value in params.items()):
            candidate_folder = os.path.join(experiments_folder, f"experiment_{experiment['id']}")
            if os.path.exists(candidate_folder):
                print(f'Parameters already exist for experiment: {candidate_folder}')
                return candidate_folder

    # Determine the next experiment number
    next_experiment_number = max((experiment['id'] for experiment in experiments), default=0) + 1

    # Create a new folder for the next experiment
    new_experiment_folder = os.path.join(experiments_folder, f'experiment_{next_experiment_number}')
    os.makedirs(new_experiment_folder, exist_ok=True)

    # Add the new experiment to the list and save to JSON file
    new_experiment = {
        'id': next_experiment_number,
        'parameters': params
    }
    experiments.append(new_experiment)
    with open(json_file, mode='w') as file:
        json.dump(experiments, file, indent=4)

    print(f'New experiment setup complete: {new_experiment_folder}')
    print(f'Parameters saved to {json_file}.')

    return new_experiment_folder

## Add metrics

In [5]:
#| export
def convert_numpy_types(obj):
    """
    Recursively convert numpy types to native Python types for JSON serialization.
    """
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    else:
        return obj

In [6]:
#| export
def add_experiment_metrics(experiments_folder: str,                    # Path to the folder containing all experiments.
                           params: Optional[Dict[str, Any]] = None,    # Optional dictionary of parameters identifying the experiment.
                           experiment_id: Optional[int] = None,        # Optional ID to identify the experiment.
                           metrics: Optional[Dict[str, Any]] = None,   # Optional dictionary of metrics to be added to the experiment.
                           json_file: Optional[str] = None             # Optional path to the JSON file tracking experiment parameters and metrics.
                          ) -> None:
    """
    Adds metrics to an existing experiment in the JSON file based on the given parameters or ID.
    """
    # Ensure the experiments folder exists
    if not os.path.exists(experiments_folder):
        raise FileNotFoundError(f"The experiments folder '{experiments_folder}' does not exist.")

    # Default JSON file to 'experiments.json' in the experiments_folder if not provided
    if json_file is None:
        json_file = os.path.join(experiments_folder, 'experiments.json')

    if not os.path.isfile(json_file):
        raise FileNotFoundError(f"The JSON file '{json_file}' does not exist.")

    if params is None and experiment_id is None:
        raise ValueError("Either 'params' or 'experiment_id' must be provided to identify the experiment.")

    if metrics is None:
        metrics = {}

    # Load existing experiments from the JSON file
    with open(json_file, mode='r') as file:
        try:
            experiments = json.load(file)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error reading JSON file: {e}")

    found_experiment = False

    # Find the matching experiment and update metrics
    for experiment in experiments:
        if (experiment_id is not None and experiment['id'] == experiment_id) or \
           (params is not None and all(experiment['parameters'].get(key) == value for key, value in params.items())):
            # Convert numpy types to native Python types before updating metrics
            experiment.update(convert_numpy_types(metrics))
            found_experiment = True
            break

    if not found_experiment:
        if experiment_id is not None:
            raise ValueError(f"Experiment with the specified ID {experiment_id} does not exist.")
        else:
            raise ValueError("Experiment with the specified parameters does not exist.")

    # Save the updated experiments back to the JSON file
    with open(json_file, mode='w') as file:
        json.dump(experiments, file, indent=4)

    if experiment_id is not None:
        print(f'Metrics added to experiment with ID {experiment_id} in {json_file}.')
    else:
        experiment_id = experiment['id']
        print(f'Metrics added to experiment with ID {experiment_id} in {json_file}.')


## Get metrics

In [7]:
#| export
def get_experiment_parameters(experiments_folder: str,                    # Path to the folder containing all experiments.
                              experiment_id: int,                         # ID to identify the experiment.
                              json_file: Optional[str] = None             # Optional path to the JSON file tracking experiment parameters and metrics.
                             ) -> Dict[str, Any]:
    """
    Retrieves the parameters of an experiment from the JSON file based on the given ID.
    """
    # Ensure the experiments folder exists
    if not os.path.exists(experiments_folder):
        raise FileNotFoundError(f"The experiments folder '{experiments_folder}' does not exist.")

    # Default JSON file to 'experiments.json' in the experiments_folder if not provided
    if json_file is None:
        json_file = os.path.join(experiments_folder, 'experiments.json')

    if not os.path.isfile(json_file):
        raise FileNotFoundError(f"The JSON file '{json_file}' does not exist.")

    # Load existing experiments from the JSON file
    with open(json_file, mode='r') as file:
        experiments = json.load(file)

    # Find the matching experiment and return its parameters
    for experiment in experiments:
        if experiment['id'] == experiment_id:
            return experiment.get('parameters', {})

    # If the experiment is not found, raise an error
    raise ValueError(f"Experiment with the specified ID {experiment_id} does not exist.")

## Convert notebook

In [8]:
#| export
def convert_notebook(notebook_path: str,                # The path to the notebook to convert.
                     output_folder: str,                # The folder to save the converted file.
                     output_filename: str,              # The name of the output file.
                     format: str = 'html'               # The format to convert the notebook to ('html' or 'pdf').
                    ) -> None:                          # This function does not return a value.
    """
    Convert the specified Jupyter notebook to HTML or PDF.

    :param notebook_path: The path to the notebook to convert.
    :param output_folder: The folder to save the converted file.
    :param output_filename: The name of the output file.
    :param format: The format to convert the notebook to ('html' or 'pdf').
    """
    if format == 'pdf' and shutil.which('pandoc') is None:
        raise RuntimeError("Pandoc is required for PDF conversion but was not found. Please install Pandoc: https://pandoc.org/installing.html")

    # Create the full path for the output file
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f"{output_filename}.{format}")

    # Convert the notebook using nbconvert
    command = f"jupyter nbconvert --to {format} \"{notebook_path}\" --output \"{output_path}\""
    try:
        subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        print(f"Notebook converted to {format.upper()} and saved at {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while converting the notebook to {format.upper()}:")
        print(e.stderr)
        raise

In [9]:
#| export
def read_json_to_dataframe(json_path: str) -> pd.DataFrame:
    """
    Reads a JSON file containing experiment results and returns a DataFrame.

    Args:
    - json_path (str): The path to the JSON file.

    Returns:
    - pd.DataFrame: A DataFrame containing the experiment results.
    """
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    # Extract relevant information
    records = []
    for item in data:
        # Assuming each item is a dictionary with an 'id' field and other details
        record = {'id_experiment': item['id']}
        record.update(item)
        records.append(record)
    
    # Create DataFrame
    df = pd.DataFrame(records)
    return df

In [10]:
#| export
def create_experiment_image_grid(experiments_folder, image_suffix, crop_length, font_size=12, save_path=None, grid_size=(3, 2), experiment_indices=None, hspace=-0.37):
    if experiment_indices is None:
        experiment_indices = [1, 2, 3, 4, 5, 6]  # Default set of indices

    # Set the directory for experiments
    experiments = [d for d in os.listdir(experiments_folder) if os.path.isdir(os.path.join(experiments_folder, d)) and 'experiment_' in d]
    experiments.sort()  # Sorting to maintain numerical order
    
    # Set up the plot with dynamic grid sizing
    fig, axes = plt.subplots(*grid_size, figsize=(5 * grid_size[1], 5 * grid_size[0]))  # Adjusted figsize dynamically
    axes = axes.flatten()
    
    for ax in axes:
        ax.axis('off')  # Hide axes

    # Process each experiment folder
    max_images = grid_size[0] * grid_size[1]
    processed_images = 0
    for idx in experiment_indices:
        if processed_images >= max_images:
            break
        experiment_name = f'experiment_{idx}'
        image_path = os.path.join(experiments_folder, experiment_name, 'images', f'exp{idx}_{image_suffix}')
        if os.path.exists(image_path):
            # Load and crop the image
            img = Image.open(image_path)
            img = img.crop((crop_length, crop_length, img.width - crop_length, img.height - crop_length))
            
            # Draw label
            draw = ImageDraw.Draw(img)
            try:
                font = ImageFont.truetype("arial.ttf", font_size)
            except IOError:
                font = ImageFont.load_default()
            text = f"Experiment {idx}"
            draw.text((10, 10), text, font=font, fill=(255, 255, 255))

            # Show image in grid
            axes[processed_images].imshow(img)
            axes[processed_images].set_title(text)
            processed_images += 1
        else:
            axes[processed_images].text(0.5, 0.5, 'Image not found', horizontalalignment='center', verticalalignment='center')
            processed_images += 1

    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(hspace=hspace)  # Reduce vertical spacing

    # Save the grid if save_path is provided
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')

    # Display the grid
    plt.show()

In [11]:
#| export
def plot_corr_matrix(dataframe: pd.DataFrame, figsize=(14, 10), cmap='coolwarm', save_path: Optional[str] = None):
    """
    Plots a correlation matrix heatmap with annotations.
    
    Parameters:
    dataframe (pd.DataFrame): The DataFrame containing the data to be analyzed.
    figsize (tuple): The size of the figure (width, height).
    cmap (str): The color map to be used for the heatmap.
    save_path (Optional[str]): The path to save the plot image. If None, the plot is not saved.
    
    Returns:
    None: Displays the correlation matrix heatmap.
    """
    # Calculate the correlation matrix
    corr_matrix = dataframe.corr()

    # Set up the matplotlib figure
    plt.figure(figsize=figsize)

    # Draw the heatmap with the correlation numbers
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap=cmap, cbar=True, linewidths=.5, square=True)

    # Set the title
    plt.title('Correlation Matrix of Metrics')

    # Save the plot if a save path is provided
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")

    # Show the plot
    plt.show()

In [12]:
#| hide
import nbdev; nbdev.nbdev_export()